In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
image_folder = '/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/'
excel_path = '/kaggle/input/eye-images-and-hb-level/Eye_Features.xlsx'

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [4]:
df = pd.read_excel(excel_path)

In [6]:
# Ensure image names match available files
df['image_path'] = df['Image_Name'].apply(lambda x: os.path.join(image_folder, x) if os.path.exists(os.path.join(image_folder, x)) else np.nan)
df.dropna(inplace=True)  # Remove rows with missing image paths

In [8]:
# Load images and labels
IMG_SIZE = (224, 224)
X, y = [], []
for _, row in df.iterrows():
    img = load_img(row['image_path'], target_size=IMG_SIZE)
    img = img_to_array(img) / 255.0  # Normalize
    X.append(img)
    y.append(row['Hb Value'])

In [9]:
X = np.array(X)
y = np.array(y)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Data augmentation
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
val_datagen = ImageDataGenerator()

In [11]:
# Load pretrained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze pretrained layers

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [12]:
# Add regression head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='linear')(x)  # Regression output

In [13]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

In [14]:
# Train model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
    epochs=50,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 92.9273 - mae: 9.4383 - val_loss: 47.0355 - val_mae: 6.6048
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - loss: 37.5088 - mae: 5.7433 - val_loss: 10.4668 - val_mae: 2.7159
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - loss: 8.1502 - mae: 2.3334 - val_loss: 3.8397 - val_mae: 1.6569
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - loss: 4.7444 - mae: 1.7622 - val_loss: 4.9105 - val_mae: 1.8583
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step - loss: 4.8760 - mae: 1.8247 - val_loss: 3.4444 - val_mae: 1.5652
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 149ms/step - loss: 4.3164 - mae: 1.7055 - val_loss: 3.6926 - val_mae: 1.5553
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - loss: 3.8063 - mae: 1.5725 - val_loss: 3.4270 - val_mae: 1.5429
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - loss: 3.8582 - mae: 1.5373 - val_loss: 3.4639 - val_mae: 1.5727
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - loss: 3.725

In [15]:
# Evaluate model
predictions = model.predict(X_val)
mae = mean_absolute_error(y_val, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step 
Mean Absolute Error: 1.55
